In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from statsmodels.formula.api import ols, glm
from statsmodels.genmod.families import Poisson


In [ ]:
# Manually set the path to the directory where the file is located
current_working_dir = #"....../ATAS-main/Stat_csv_files/" #Change path as required
os.chdir(current_working_dir)  # Set the working directory

# Read the CSV file
xdata_pre = pd.read_csv("AWNS_AWS_all_details.csv")  # Assuming data.csv exists in the specified directory

In [ ]:
xdata_pre = xdata_pre.rename(columns={
    'Mean Pause_s': 'Mean_Pause_s',
    'Mean Speech_s': 'Mean_Speech_s',
    'CV Pause': 'CV_Pause',
    'CV Speech': 'CV_Speech',
})

In [ ]:
# List of columns to convert
columns_to_convert = ['Mean_Speech_s', 'Mean_Pause_s', 'long_p_durations_mean', 'short_p_durations_mean']

# Convert from seconds to milliseconds and create new columns
for col in columns_to_convert:
    xdata_pre[f"{col}_ms"] = xdata_pre[col] * 1000

In [ ]:
# Define variables for continuous and count models
continuous_metrics = {
    "Speech_Rate": "Speech Rate",
    "Pause_Duration_s": "Pause Duration (s)",
    "Mean_Pause_s_ms": "Mean Pause Duration (ms)",
    "Mean_Speech_s_ms": "Mean Speech Duration (ms)",
    "CV_Pause": "Coefficient of Variation - Pause",
    "CV_Speech": "Coefficient of Variation - Speech",
    "long_p_durations_mean_ms": "Mean Long Pause Duration (ms)",
    "short_p_durations_mean_ms": "Mean Short Pause Duration (ms)",
    "long_p_durations_cv": "CV of Long Pauses",
    "short_p_durations_cv": "CV of Short Pauses"
}

count_metrics = {
    "Pause_Events": "Number of Pause Events",
    "long_p_count": "Long Pause Events",
    "short_p_count": "Short Pause Events"
}

In [ ]:
xdata = xdata_pre.copy()

## Stats for Groups

In [ ]:
# Define a function to calculate the statistics for continuous and count metrics
def calculate_statistics(df, group_col, metrics):
    # Create an empty list to hold the individual metric DataFrames
    metric_stats = []
    
    for metric, name in metrics.items():
        # Calculate the statistics for each metric in the group
        group_stats = df.groupby(group_col)[metric].agg(
            mean='mean',
            std='std',
            max='max',
            min='min'                       
        )
        # Format to 3 decimal places
        group_stats = group_stats.round(3)
        # Rename the columns to match the friendly names
        group_stats.columns = [f'{name}_mean',   f'{name}_std',f'{name}_max',f'{name}_min',]
        
        metric_stats.append(group_stats)
    
    # Concatenate all the metric stats into a single DataFrame
    return pd.concat(metric_stats, axis=1)

# Assume xdata is your dataframe and 'group_column' is the column for AWS and AWNS grouping
group_column = 'Group'  # Adjust this if the group column name is different in your data

# Calculate statistics for continuous metrics
continuous_stats = calculate_statistics(xdata, group_column, continuous_metrics)

# Calculate statistics for count metrics
count_stats = calculate_statistics(xdata, group_column, count_metrics)

# Combine both continuous and count statistics into a final table
final_table = pd.concat([continuous_stats, count_stats], axis=1)


In [ ]:
# Adjust pandas display options to avoid clipping of the table
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows (if the table is not too large)
pd.set_option('display.width', None)        # Disable line wrapping
pd.set_option('display.max_colwidth', None) # Disable column width truncation

# Now, display the final_table
final_table


,Speech Rate_mean,Speech Rate_std,Speech Rate_max,Speech Rate_min,Pause Duration (s)_mean,Pause Duration (s)_std,Pause Duration (s)_max,Pause Duration (s)_min,Mean Pause Duration (ms)_mean,Mean Pause Duration (ms)_std,Mean Pause Duration (ms)_max,Mean Pause Duration (ms)_min,Mean Speech Duration (ms)_mean,Mean Speech Duration (ms)_std,Mean Speech Duration (ms)_max,Mean Speech Duration (ms)_min,Coefficient of Variation - Pause_mean,Coefficient of Variation - Pause_std,Coefficient of Variation - Pause_max,Coefficient of Variation - Pause_min,Coefficient of Variation - Speech_mean,Coefficient of Variation - Speech_std,Coefficient of Variation - Speech_max,Coefficient of Variation - Speech_min,Mean Long Pause Duration (ms)_mean,Mean Long Pause Duration (ms)_std,Mean Long Pause Duration (ms)_max,Mean Long Pause Duration (ms)_min,Mean Short Pause Duration (ms)_mean,Mean Short Pause Duration (ms)_std,Mean Short Pause Duration (ms)_max,Mean Short Pause Duration (ms)_min,CV of Long Pauses_mean,CV of Long Pauses_std,CV of Long Pauses_max,CV of Long Pauses_min,CV of Short Pauses_mean,CV of Short Pauses_std,CV of Short Pauses_max,CV of Short Pauses_min,Number of Pause Events_mean,Number of Pause Events_std,Number of Pause Events_max,Number of Pause Events_min,Long Pause Events_mean,Long Pause Events_std,Long Pause Events_max,Long Pause Events_min,Short Pause Events_mean,Short Pause Events_std,Short Pause Events_max,Short Pause Events_min
Group,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AWNS,163.118,23.427,201.911,120.053,16.477,4.680,24.47,10.552,286.514,62.153,429.137,197.403,1258.144,226.640,1758.194,863.545,0.890,0.113,1.089,0.708,0.738,0.065,0.859,0.625,450.992,75.424,620.455,330.480,86.007,5.361,94.000,76.000,0.533,0.084,0.715,0.408,0.326,0.034,0.386,0.266,58.000,12.966,80,35,31.111,7.226,46,21,26.889,8.737,47,14
AWS,127.595,38.106,180.847,53.553,30.681,22.398,76.88,10.079,338.136,84.395,519.090,245.804,1250.822,358.108,1966.417,751.382,0.902,0.163,1.231,0.659,0.823,0.099,0.975,0.612,484.015,108.228,701.176,359.219,90.407,6.572,99.682,78.292,0.613,0.177,0.957,0.401,0.313,0.035,0.357,0.225,86.471,56.367,250,35,55.118,38.537,168,22,31.353,18.347,82,13


## GLM Models Assumptions Check

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import shapiro, skew, kurtosis
from statsmodels.stats.diagnostic import het_breuschpagan

def check_glm_assumptions_stats(xdata, continuous_metrics, count_metrics):
    predictors = ["Group", "Sex", "Age"]
    
    # Convert categorical variables to dummies for VIF calculation
    X = xdata[predictors].copy()
    X = pd.get_dummies(X, drop_first=True)

    # 1. Compute VIF for Predictors
    print("\n### Checking Multicollinearity (VIF) ###")
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    
    print(vif_data)
    print("\nInterpretation: VIF > 10 indicates severe multicollinearity. Consider removing/reducing correlated predictors.")

    # 2. Check Statistical Properties of Continuous Variables
    print("\n### Statistical Summary of Continuous Variables (Before and After Log Transformation) ###")
    
    for metric in continuous_metrics.keys():
        log_metric = f'log_{metric}'
        xdata[log_metric] = np.log1p(xdata[metric])  # Log transformation (if applicable)

        # Compute summary statistics before transformation
        mean_val_orig = np.mean(xdata[metric])
        std_val_orig = np.std(xdata[metric])
        skewness_orig = skew(xdata[metric])
        kurt_orig = kurtosis(xdata[metric])

        print(f"\n{metric} (Original):")
        print(f"Mean: {mean_val_orig:.4f}, Std Dev: {std_val_orig:.4f}, Skewness: {skewness_orig:.4f}, Kurtosis: {kurt_orig:.4f}")

        # Shapiro-Wilk Test before transformation
        shapiro_test_orig = shapiro(xdata[metric])
        print(f"Shapiro-Wilk Test (Original): W = {shapiro_test_orig.statistic:.4f}, p-value = {shapiro_test_orig.pvalue:.4f}")

        if shapiro_test_orig.pvalue < 0.05:
            print("  -> Data is NOT normally distributed (p < 0.05). Consider transformation.")
            
            # Compute summary statistics after transformation
            mean_val_log = np.mean(xdata[log_metric])
            std_val_log = np.std(xdata[log_metric])
            skewness_log = skew(xdata[log_metric])
            kurt_log = kurtosis(xdata[log_metric])

            print(f"\n{log_metric} (Log-Transformed):")
            print(f"Mean: {mean_val_log:.4f}, Std Dev: {std_val_log:.4f}, Skewness: {skewness_log:.4f}, Kurtosis: {kurt_log:.4f}")

            # Shapiro-Wilk Test after transformation
            shapiro_test_log = shapiro(xdata[log_metric])
            print(f"Shapiro-Wilk Test (Log-Transformed): W = {shapiro_test_log.statistic:.4f}, p-value = {shapiro_test_log.pvalue:.4f}")

            if shapiro_test_log.pvalue < 0.05:
                print("  -> Data is STILL not normally distributed (p < 0.05). Consider alternative transformations or robust methods.")
            else:
                print("  -> Log transformation improved normality (p > 0.05).")
        else:
            print("  -> Data appears normally distributed (p > 0.05). Skipping log-transformation test.")

    # 3. Breusch-Pagan Test for Homoscedasticity (Using Original Data)
    print("\n### Homoscedasticity Check (Breusch-Pagan Test) ###")

    for metric in continuous_metrics.keys():
        formula = f'{metric} ~ Group + Sex + Age'  # Use original variable, not log-transformed
        model = smf.glm(formula, data=xdata, family=sm.families.Gaussian()).fit()

        residuals = model.resid_deviance
        fitted_values = model.fittedvalues

        bp_test = het_breuschpagan(residuals, sm.add_constant(fitted_values))
        print(f"\n{metric} - Breusch-Pagan Test:")
        print(f"LM Statistic: {bp_test[0]:.4f}, p-value: {bp_test[1]:.4f}")

        if bp_test[1] < 0.05:
            print("  -> Residual variance is NOT constant (heteroscedasticity detected). Consider weighted regression.")
        else:
            print("  -> Residual variance appears constant (homoscedasticity).")
        
    print("\n### Checking Overdispersion in Poisson Models ###")
    for metric in count_metrics.keys():
        formula = f'{metric} ~ Group + Sex + Age'
        model = smf.glm(formula, data=xdata, family=sm.families.Poisson()).fit()
        
        pearson_chi2 = sum((model.resid_pearson)**2)
        df_residual = model.df_resid
        dispersion_ratio = pearson_chi2 / df_residual
        print(f"{metric}: Dispersion Ratio = {dispersion_ratio}")
        
        if dispersion_ratio > 1.5:
            print(f"WARNING: {metric} shows signs of overdispersion (Dispersion Ratio > 1.5). Consider using a Negative Binomial model.")

# Run statistical assumption checks
check_glm_assumptions_stats(xdata, continuous_metrics, count_metrics)



### Checking Multicollinearity (VIF) ###
    Variable       VIF
0        Age  3.354182
1  Group_AWS  2.182836
2      Sex_M  2.538846

Interpretation: VIF > 10 indicates severe multicollinearity. Consider removing/reducing correlated predictors.

### Statistical Summary of Continuous Variables (Before and After Log Transformation) ###

Speech_Rate (Original):
Mean: 145.8638, Std Dev: 35.2932, Skewness: -0.8249, Kurtosis: 0.2940
Shapiro-Wilk Test (Original): W = 0.9423, p-value = 0.0658
  -> Data appears normally distributed (p > 0.05). Skipping log-transformation test.

Pause_Duration_s (Original):
Mean: 23.3761, Std Dev: 17.0401, Skewness: 2.0611, Kurtosis: 3.3849
Shapiro-Wilk Test (Original): W = 0.6903, p-value = 0.0000
  -> Data is NOT normally distributed (p < 0.05). Consider transformation.

log_Pause_Duration_s (Log-Transformed):
Mean: 3.0311, Std Dev: 0.5201, Skewness: 1.1656, Kurtosis: 0.5725
Shapiro-Wilk Test (Log-Transformed): W = 0.8711, p-value = 0.0007
  -> Data is STILL 

## GLM Models

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

def model_continuous_metrics(xdata, continuous_metrics):
    predictors = ["Group", "Sex", "Age"]
    
    # Define metrics for log transformation
    transformed_metrics = {
        "Mean_Pause_s_ms": "log_Mean_Pause_s_ms"
    }
    
    # Define weighted metrics and their specific families
    weighted_metrics = {
        "Speech_Rate": sm.families.Gaussian(),  # Weighted GLM for Speech_Rate (Gaussian family)
        "Pause_Duration_s": sm.families.Gamma(link=sm.families.links.log()),  # Weighted GLM with Gamma for Pause_Duration_s
        "long_p_durations_cv": sm.families.Gamma(link=sm.families.links.log())  # Weighted GLM with Gamma for long_p_durations_cv
    }

    # Define default family for GLM Gaussian (for the remaining metrics)
    default_family = sm.families.Gaussian()

    # Convert continuous_metrics list to dictionary
    updated_metrics = {metric: metric for metric in continuous_metrics}

    # Apply log transformation where necessary
    for orig_var, log_var in transformed_metrics.items():
        if orig_var in updated_metrics:
            xdata[log_var] = np.log1p(xdata[orig_var])  # Apply log transformation
            updated_metrics[log_var] = updated_metrics.pop(orig_var)  # Update dictionary key
    
    results = []
    
    for metric in updated_metrics.keys():
        formula = f'{metric} ~ ' + ' + '.join(predictors)
        
        # Check if the metric is in the weighted metrics dictionary
        if metric in weighted_metrics:
            model = smf.glm(formula, data=xdata, family=weighted_metrics[metric]).fit()
            model_type = "Weighted GLM"
        else:
            # For all other metrics, use Gaussian GLM (except those needing specific transformations)
            model = smf.glm(formula, data=xdata, family=default_family).fit()
            model_type = "GLM"
        
        # Collect the results
        summary_df = pd.DataFrame({
            "Metric": [metric] * len(model.params),
            "Predictor": model.params.index,
            "Coefficient": model.params.values,
            "Std Error": model.bse.values,
            "z-value": model.tvalues.values,
            "p-value": model.pvalues.values,
            "Model Type": [model_type] * len(model.params)  # GLM or Weighted GLM
        })
        
        results.append(summary_df)
    
    # Combine all results into one DataFrame
    final_results_df = pd.concat(results, ignore_index=True)
    return final_results_df


In [ ]:
results_df = model_continuous_metrics(xdata, continuous_metrics)

## Count data negative binomial models

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def fit_standard_negative_binomial(xdata, count_metrics):
    results = []
    
    # Set the predictors you want to include
    predictors = ["Group", "Sex", "Age"]
    
    for metric in count_metrics.keys():
        # Ensure the response variable is numeric
        y = pd.to_numeric(xdata[metric], errors='coerce')

        # Ensure predictor variables are numeric or categorical
        X = xdata[predictors].copy()
        
        # Convert categorical variables to dummy variables
        X = pd.get_dummies(X, columns=["Group", "Sex"], drop_first=True)  
        # "AWS" will be the reference for Group, "M" will be the reference for Sex

        # Add a constant term
        X = sm.add_constant(X)

        # Drop NaN values if necessary
        X = X.dropna()
        y = y.loc[X.index]  # Ensure y matches X after dropping NaNs

        # Check if X is empty before fitting the model
        if X.shape[0] == 0:
            print(f"Warning: No valid data available for {metric}. Skipping.")
            continue
        
        # Fit the standard Negative Binomial model
        model = sm.NegativeBinomial(y, X).fit()
        
        # Get the summary of the model fit
        summary = model.summary2().tables[1]
        
        # Find the correct standard error column
        std_err_col = [col for col in summary.columns if 'Std' in col][0]
        
        # Collect results into a list
        for index, row in summary.iterrows():
            results.append({
                "Metric": metric,
                "Predictor": index,
                "Coefficient": row["Coef."],
                "Std Error": row[std_err_col],
                "z-value": row["z"],
                "p-value": row["P>|z|"],
                "Model Type": "Negative Binomial",
            })
    
    results_df_count = pd.DataFrame(results)
    return results_df_count


In [ ]:
results_df_count = fit_standard_negative_binomial(xdata,count_metrics)

Optimization terminated successfully.
         Current function value: 4.712794
         Iterations: 13
         Function evaluations: 21
         Gradient evaluations: 21
Optimization terminated successfully.
         Current function value: 4.253274
         Iterations: 13
         Function evaluations: 21
         Gradient evaluations: 21
Optimization terminated successfully.
         Current function value: 3.822291
         Iterations: 13
         Function evaluations: 19
         Gradient evaluations: 19


In [ ]:
all_results = pd.concat([results_df, results_df_count])

In [ ]:
all_results

,Metric,Predictor,Coefficient,Std Error,z-value,p-value,Model Type
0,Speech_Rate,Intercept,166.296120,17.102439,9.723533,2.393312e-22,Weighted GLM
1,Speech_Rate,Group[T.AWS],-34.805783,11.559599,-3.010985,2.604015e-03,Weighted GLM
2,Speech_Rate,Sex[T.M],4.609714,11.471370,0.401845,6.877980e-01,Weighted GLM
3,Speech_Rate,Age,-0.180599,0.456953,-0.395225,6.926769e-01,Weighted GLM
4,Pause_Duration_s,Intercept,2.678939,0.302814,8.846815,9.005277e-19,Weighted GLM
5,Pause_Duration_s,Group[T.AWS],0.587942,0.204673,2.872592,4.071195e-03,Weighted GLM
6,Pause_Duration_s,Sex[T.M],0.127129,0.203111,0.625909,5.313748e-01,Weighted GLM
7,Pause_Duration_s,Age,0.001618,0.008091,0.199930,8.415351e-01,Weighted GLM
8,Mean_Speech_s_ms,Intercept,1422.388965,141.232530,10.071256,7.402663e-24,GLM
9,Mean_Speech_s_ms,Group[T.AWS],37.205355,95.459570,0.389750,6.967215e-01,GLM


In [ ]:
all_results_filtered = all_results[(all_results["Predictor"] == "Group[T.AWS]") | 
    (all_results["Predictor"] == "Group_AWS")]

In [ ]:
file_path = #'.../pval.csv' # Change as required
all_results_filtered.to_csv(file_path, index=False)

print(f"CSV file saved at: {file_path}")

In [ ]:
all_results_filtered

,Metric,Predictor,Coefficient,Std Error,z-value,p-value,Model Type
1,Speech_Rate,Group[T.AWS],-34.805783,11.559599,-3.010985,0.002604,Weighted GLM
5,Pause_Duration_s,Group[T.AWS],0.587942,0.204673,2.872592,0.004071,Weighted GLM
9,Mean_Speech_s_ms,Group[T.AWS],37.205355,95.459570,0.389750,0.696722,GLM
13,CV_Pause,Group[T.AWS],-0.004023,0.050537,-0.079596,0.936559,GLM
17,CV_Speech,Group[T.AWS],0.089461,0.028948,3.090369,0.001999,GLM
21,long_p_durations_mean_ms,Group[T.AWS],29.997968,33.860736,0.885922,0.375660,GLM
25,short_p_durations_mean_ms,Group[T.AWS],4.873901,2.194005,2.221463,0.026320,GLM
29,long_p_durations_cv,Group[T.AWS],0.129545,0.082816,1.564242,0.117761,Weighted GLM
33,short_p_durations_cv,Group[T.AWS],-0.006305,0.012138,-0.519417,0.603470,GLM
37,log_Mean_Pause_s_ms,Group[T.AWS],0.164051,0.081611,2.010168,0.044413,GLM


## Testing Benjamini Hochberg

In [ ]:
p_values = all_results_filtered["p-value"].to_numpy()

In [ ]:
import numpy as np

def benjamini_hochberg(p_values, fdr=0.1):
    """
    Perform the Benjamini-Hochberg procedure for controlling the false discovery rate.
    
    Parameters:
    - p_values (list or np.array): Array of p-values to correct.
    - fdr (float): Desired false discovery rate.
    
    Returns:
    - np.array: Boolean array representing whether each test is significant.
    """
    p_values = np.array(p_values)
    n = len(p_values)
    sorted_indices = np.argsort(p_values)
    sorted_p_values = p_values[sorted_indices]
    thresholds = fdr * np.arange(1, n + 1) / n
    
    reject = sorted_p_values <= thresholds
    reject_max_index = np.where(reject)[0].max() if any(reject) else 0
    reject[:reject_max_index + 1] = True
    
    corrected_sorted_indices = np.argsort(sorted_indices)
    return reject[corrected_sorted_indices]


In [ ]:
is_significant = benjamini_hochberg(p_values, fdr=0.1)

print(f"P-values: {p_values}")
print(f"Significant at FDR=0.1: {is_significant}")

P-values: [2.60401484e-03 4.07119505e-03 6.96721517e-01 9.36558565e-01
 1.99908294e-03 3.75659604e-01 2.63196018e-02 1.17760737e-01
 6.03469769e-01 4.44133681e-02 1.23447953e-02 6.59099560e-04
 4.47472515e-01]
Significant at FDR=0.1: [ True  True False False  True False  True False False  True  True  True
 False]
